<a href="https://colab.research.google.com/github/SCCSMARTCODE/rag-mastery/blob/main/1_Hybrid_Search_using_weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -U bitsandbytes==0.48.1
!pip install accelerate transformers
!pip install weaviate-client
!pip install pipeline
!pip install colorama
%pip install -qU langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from transformers import pipeline

import os
import weaviate
import textwrap
import logging
from colorama import Fore, Style, init
from weaviate.classes.init import Auth
from google.colab import userdata
from weaviate.classes.config import Configure, DataType, Property, ReferenceProperty
from langchain_community.document_loaders import PyPDFLoader

In [84]:
model_path = "microsoft/UserLM-8b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
    ).to("cuda")

In [78]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",

    # generation control
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    repetition_penalty=1.1,

    # output control
    max_new_tokens=2048,
    num_return_sequences=1,

    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

Device set to use cuda:0


In [56]:
# result = pipe("What is RAG")
# print(result)
# print(result[0]['generated_text'])

In [25]:
file_path = "docs.pdf"
loader = PyPDFLoader(file_path)

In [28]:
docs = loader.load()

In [57]:
# docs[0].model_dump_json()

In [49]:
# Best practice: store your credentials in environment variables
weaviate_url = userdata.get('WEAVIATE_URL')
weaviate_api_key = userdata.get('WEAVIATE_API_KEY')
hf_api_key = userdata.get('HF_TOKEN')

headers = {
    "X-HuggingFace-Api-Key": hf_api_key,
}

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers=headers
)


print(client.is_ready())

True


In [22]:
client.collections.list_all()

{}

In [45]:
# --- Parent Collection ---
client.collections.create(
    name="Files",
    description="RAG Collection",
    vectorizer_config=Configure.Vectorizer.text2vec_huggingface(
        model="sentence-transformers/all-MiniLM-L6-v2"
    ),
    properties=[
        Property(name="source", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="author", data_type=DataType.TEXT),
        Property(name="total_pages", data_type=DataType.INT),
    ],

)


# --- Child Collection ---
client.collections.create(
    name="Pages",
    vectorizer_config=Configure.Vectorizer.text2vec_huggingface(
        model="sentence-transformers/all-MiniLM-L6-v2"
    ),
    properties=[
        Property(name="content", data_type=DataType.TEXT),
        Property(name="page", data_type=DataType.INT),
    ],
    references=[  # ✅ references go here, not in properties
        ReferenceProperty(name="source_ref", target_collection="Files"),
    ],
)

In [55]:
client.collections.list_all()

{'Files': _CollectionConfigSimple(name='Files', description='RAG Collection', generative_config=None, properties=[_Property(name='source', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-huggingface', vectorizer_configs=None), _Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-huggingface', vectorizer_configs=None), _Property(name='author', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested

In [51]:
files = client.collections.get("Files")
pages = client.collections.get("Pages")

In [52]:
file_obj = files.data.insert(
    properties={
        "source": file_path,
        "title": "Retrieval-Augmented Generation",
        "author": "Patrick Lewis et al.",
        "total_pages": 19
    }
)

In [54]:
for doc in docs:
    pages.data.insert(
        properties={
            "content": doc.page_content,
            "page": doc.metadata['page'],
        },
    references={
        "source_ref": file_obj
    }
    )

In [67]:

init(autoreset=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)
log = logging.getLogger("RAG-Pipeline")

In [73]:
def retrieve_hybrid(query: str, k: int = 5, alpha: float = 0.5):
    """Return list of (score, properties) from Weaviate hybrid query, with print debugging."""
    print(f"{Fore.CYAN}\n🔍 Retrieving from Weaviate:", query)
    resp = pages.query.hybrid(query=query, alpha=alpha, limit=k)

    hits = []
    objs = getattr(resp, "objects", None) or []

    if not objs:
        print(f"{Fore.YELLOW}⚠️  No results returned from Weaviate.")
        return hits

    print(f"{Fore.GREEN}✅ Retrieved {len(objs)} candidates:\n")
    for i, o in enumerate(objs, 1):
        props = getattr(o, "properties", {}) or getattr(o, "__dict__", {}).get("properties", {})
        score = (
            getattr(o, "score", None)
            or getattr(o, "certainty", None)
            or getattr(o, "distance", None)
        )
        hits.append((score, props))

        snippet = (props.get("content") or props.get("body") or "")[:250].replace("\n", " ")
        title = props.get("title") or props.get("source") or "Untitled"
        print(
            f"{Fore.LIGHTBLUE_EX}[{i}] {Style.BRIGHT}{title}{Style.RESET_ALL} "
            f"| score={score} | page={props.get('page')}\n"
            f"    {snippet}..."
        )

    return hits

In [75]:
def build_context(hits, max_chars=3000):
    pieces = []
    for i, (_, props) in enumerate(hits, 1):
        title = props.get("title") or props.get("source") or ""
        page = props.get("page")
        content = props.get("content") or props.get("body") or ""
        header = f"[{i}] {title} (page {page})"
        pieces.append(header + "\n" + content.strip())
    ctx = "\n\n---\n\n".join(pieces)
    # truncate conservatively to avoid hitting token limits
    return ctx if len(ctx) <= max_chars else ctx[:max_chars]

In [80]:
def rag_answer(question: str, k=5, alpha=0.5, max_new_tokens=2048):
    print(f"{Fore.MAGENTA}\n🧠 Question:", question)
    hits = retrieve_hybrid(question, k=k, alpha=alpha)
    if not hits:
        print(f"{Fore.RED}❌ No relevant documents found.")
        return "No relevant documents found."

    context = build_context(hits)
    prompt = textwrap.dedent(f"""
        You are an assistant that answers using only the provided context below.
        If the answer is not in the context, say "I don't know".
        Cite sources using bracket numbers like [1], [2].

        Context:
        {context}

        Question: {question}

        Answer and cite sources.
    """).strip()

    print(f"{Fore.YELLOW}\n📄 Constructed Prompt:\n{'-'*80}\n{prompt[:800]}...\n{'-'*80}")
    out = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    generated = out[0]["generated_text"] if isinstance(out, list) else out["generated_text"]

    print(f"{Fore.CYAN}\n💬 Model Output:\n{'-'*80}\n{generated}\n{'-'*80}")
    return generated

In [82]:
answer = rag_answer("What is Retrieval-Augmented Generation (RAG)?", k=4, alpha=0.5)
print("\nFinal Answer:\n", answer)